# Udacity - Fundamentos de Data Science II
---
### Módulo Data Wrangle

#### Suzana Barboza

# Projeto We Rate Dogs
---


Projeto com ênfase na prática do aprendizado de Data Wrangle. Feito na linguagem Python.
É feita a coleta de 3 base de dados distintas e a limpeza das mesmas. Após isso, é feita a junção dos dados de acordo com a lei dos dados arrumados.
Para a conclusão é feita visualizações da base de dados final


## 1. Gather 

In [284]:
#importação de bibliotecas necessárias
import pandas as pd
import numpy as np
import requests
import tweepy
import json
from datetime import datetime as dt

### 1.1 CSV

Busca do dataframe na planilha fornecida pela Udacity

In [285]:
df_wrd = pd.read_csv('twitter-archive-enhanced.csv') #dataframe we rate dogs

### 1.2 TSV

Busca o arquivo na url fornecida pela Udacity e armazena em um dataframe


In [286]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
rows  = response.text.split('\n')
#deleta última linha em branco
del rows[len(rows)-1]
#separa as colunas de acordo com a tabulação
for i in range(len(rows)):
    rows[i] = rows[i].split('\t')
header = rows[0]
#deleta o header
del rows[0]
df_prediction = pd.DataFrame(np.array(rows), columns=header)

### 1.3 Twitter

Buscar informações de tweets do dataframe 'we rate dogs' através da API e armazena em um arquivo de texto nomeado 'tweet_json.txt'.

Após a coleta o armazenamento desse arquivo, ele é lido e armazenado em um dataframe

In [287]:
# Pretty Print JSON method
def pp_json(json_thing, sort=True, indents=4):
    if type(json_thing) is str:
        print(json.dumps(json.loads(json_thing), sort_keys=sort, indent=indents))
    else:
        print(json.dumps(json_thing, sort_keys=sort, indent=indents))
    return None

def default_json(o):
    if isinstance(o, np.int64): return int(o)  
    raise TypeError
    
#Função que faz a busca no twitter  das informações do dataframe provido pelo parâmetro df_wrd e cria o arquivo txt em formato json   
def getInfoFromTwitter(df_wrd, consumerkey, consumer_secret, access_token, access_token_secret):
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    ids = df_wrd.tweet_id
    tweet_info = {}
    erros_count = 0
    print("Início: ", dt.now().time())
    for i in ids:    
        try:
            status = api.get_status(i) 
            tweet_info[i] = {'favorite_count': status.favorite_count, 'retweet_count':status.retweet_count }           
        except tweepy.TweepError as error:
            print("Erro.. ", error, " ", dt.now().time())        
            erros_count +=1        
    print("Total: ", len(ids))
    print("Erros: ", erros_count)
    print("Fim: ", dt.now().time())
    #write json
    with open('tweet_json.txt', 'w') as outfile:  
        json.dump(tweet_info, outfile, default=default_json)

In [288]:
#chama o método para criação do arquivo tweet_json.txt
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''
#getInfoFromTwitter(df_wrd, consumer_key, consumer_secret, access_token, access_token_secret)

In [289]:
#lê o arquivo tweet_json.txt e armazena em um dataframe
with open('tweet_json.txt') as file:
    twitter_data = json.load(file)
data = []
for key in twitter_data.keys():
    data.append([key, twitter_data[key]['favorite_count'], twitter_data[key]['retweet_count']])
header = ['twitter_id', 'favorite_count', 'retweet_count']
df_twitter = pd.DataFrame(np.array(data), columns=header)

## 2. Access



In [9]:
df_wrd.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
183,856543823941562368,NaN,NaN,2017-04-24 16:22:16 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Callie. She'll be your navigator today...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/856543823...,12,10,Callie,None,None,None,None
766,777684233540206592,NaN,NaN,2016-09-19 01:42:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...","""Yep... just as I suspected. You're not flossi...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/777684233...,12,10,None,None,None,None,None
318,834574053763584002,NaN,NaN,2017-02-23 01:22:14 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here's a doggo fully pupared for a shower. H*c...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/834574053...,13,10,None,doggo,None,None,None
744,780192070812196864,NaN,NaN,2016-09-25 23:47:39 +0000,"<a href=""http://twitter.com/download/iphone"" r...",We only rate dogs. Pls stop sending in non-can...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/780192070...,11,10,None,None,None,None,None
1372,702217446468493312,NaN,NaN,2016-02-23 19:44:20 +0000,"<a href=""http://twitter.com/download/iphone"" r...","I know it's tempting, but please stop sending ...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/702217446...,9,10,None,None,None,None,None


In [10]:
df_wrd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [11]:
df_wrd.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [12]:
df_wrd[df_wrd.in_reply_to_status_id.notna()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
30,886267009285017600,8.862664e+17,2.281182e+09,2017-07-15 16:51:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@NonWhiteHat @MayhewMayhem omg hello tanner yo...,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
55,881633300179243008,8.816070e+17,4.738443e+07,2017-07-02 21:58:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@roushfenway These are good dogs but 17/10 is ...,NaN,NaN,NaN,NaN,17,10,None,None,None,None,None
64,879674319642796034,8.795538e+17,3.105441e+09,2017-06-27 12:14:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@RealKentMurphy 14/10 confirmed,NaN,NaN,NaN,NaN,14,10,None,None,None,None,None
113,870726314365509632,8.707262e+17,1.648776e+07,2017-06-02 19:38:25 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@ComplicitOwl @ShopWeRateDogs &gt;10/10 is res...,NaN,NaN,NaN,NaN,10,10,None,None,None,None,None
148,863427515083354112,8.634256e+17,7.759620e+07,2017-05-13 16:15:35 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@Jack_Septic_Eye I'd need a few more pics to p...,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
149,863079547188785154,6.671522e+17,4.196984e+09,2017-05-12 17:12:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Ladies and gentlemen... I found Pipsy. He may ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/863079547...,14,10,None,None,None,None,None
179,857214891891077121,8.571567e+17,1.806710e+08,2017-04-26 12:48:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@Marc_IRL pixelated af 12/10,NaN,NaN,NaN,NaN,12,10,None,None,None,None,None
184,856526610513747968,8.558181e+17,4.196984e+09,2017-04-24 15:13:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...","THIS IS CHARLIE, MARK. HE DID JUST WANT TO SAY...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/856526610...,14,10,None,None,None,None,None
186,856288084350160898,8.562860e+17,2.792810e+08,2017-04-23 23:26:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@xianmcguire @Jenna_Marbles Kardashians wouldn...,NaN,NaN,NaN,NaN,14,10,None,None,None,None,None
188,855862651834028034,8.558616e+17,1.943518e+08,2017-04-22 19:15:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@dhmontgomery We also gave snoop dogg a 420/10...,NaN,NaN,NaN,NaN,420,10,None,None,None,None,None


In [14]:
df_prediction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null object
jpg_url     2075 non-null object
img_num     2075 non-null object
p1          2075 non-null object
p1_conf     2075 non-null object
p1_dog      2075 non-null object
p2          2075 non-null object
p2_conf     2075 non-null object
p2_dog      2075 non-null object
p3          2075 non-null object
p3_conf     2075 non-null object
p3_dog      2075 non-null object
dtypes: object(12)
memory usage: 194.6+ KB


In [15]:
df_prediction.describe()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
count,2075,2075,2075,2075,2075,2075,2075,2075,2075,2075,2075,2075
unique,2075,2009,4,378,2006,2,405,2004,2,408,2006,2
top,882762694511734784,https://pbs.twimg.com/media/CdHwZd0VIAA4792.jpg,1,golden_retriever,0.796313,True,Labrador_retriever,0.0693617,True,Labrador_retriever,0.000917414,True
freq,1,2,1780,150,2,1532,104,3,1553,79,2,1499


#### 2.1 Problemas de Qualidade

##### 1º Iteração

##### Tabela We Rate Dogs

- As colunas in_reply_to_status_id,in_reply_to_user_id, retweeted_status, retweeted_status_user_id  são do tipo **float** e deveriam ser do tipo **int64** 
- As colunas timestamp, retweeted_status_timestamp são do tipo **object** e deveriam ser do tipo **datetime**
- As colunas rating_numerator e rating_denomitator possuem outliers significativos levando em conta o máximo, a média e o percentis.
- As colunas doggo, floofer, pupper, puppo deveriam ser do tipo **bool**


##### Tabela de Predição

- A coluna tweet_id é do tipo **object** e deveria ser do tipo **int64**
- A coluna img_num é do tipo **object** e deveria ser do tipo **int**
- As colunas p1_dog, p2_dog, p3_dog são do tipo **object** e deveriam ser do tipo **bool**
- As colunas p1_conf, p2_conf, p3_conf são do tipo **object** e deveriam ser do tipo **float**
- O formato das colunas p1, p2, p3 deveriam ser padronizados


#### 2.2 Problemas de Arrumação

##### Tabela We Rate Dogs

- O nome das colunas timestamp, retweeted_status_timestamp deveriam ser mais intuitivo

##### Tabela Predição
- O nome das colunas jpg_url, img_num, p1, p1_conf, p1_dog, p2, p2_conf, p2_dog, p3, p3_conf, p3_dog deveria ser mais intuitivo


### 3. Clean

In [268]:
#Copy datasets df_wrd, df_prediction, df_twitter
df_wrd_copy = df_wrd.copy()
df_prediction_copy = df_prediction.copy()
df_twitter_copy = df_twitter.copy()

#### Definir

##### Tabela We Rate Dogs

- As colunas in_reply_to_status_id,in_reply_to_user_id, retweeted_status_id, retweeted_status_user_id são do tipo **float** e deveriam ser do tipo **int64**
- As colunas timestamp, retweeted_status_timestamp são do tipo **object** e deveriam ser do tipo **datetime

##### Tabela de Predição

- A coluna tweet_id é do tipo **object** e deveria ser do tipo **int64**
- A coluna img_num é do tipo **object** e deveria ser do tipo **int**
- As colunas p1_dog, p2_dog, p3_dog são do tipo **object** e deveriam ser do tipo **bool**
- As colunas p1_conf, p2_conf, p3_conf são do tipo **object** e deveriam ser do tipo **float**


#### Programar

In [269]:
df_prediction_copy.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.0614285,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.07419169999999999,True,Rhodesian_ridgeback,0.07201,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.13858399999999998,True,bloodhound,0.11619700000000001,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [270]:
df_wrd_copy.in_reply_to_status_id = df_wrd_copy.in_reply_to_status_id.notna().astype('int64')
df_wrd_copy.in_reply_to_user_id = df_wrd_copy.in_reply_to_user_id.notna().astype('int64')
df_wrd_copy.retweeted_status_id = df_wrd_copy.retweeted_status_id.notna().astype('int64')
df_wrd_copy.retweeted_status_user_id  = df_wrd_copy.retweeted_status_user_id .notna().astype('int64')

df_wrd_copy.timestamp = pd.to_datetime(df_wrd_copy.timestamp, format='%Y-%m-%d %H:%M:%S')
df_wrd_copy.retweeted_status_timestamp = pd.to_datetime(df_wrd_copy.retweeted_status_timestamp, format='%Y-%m-%d %H:%M:%S')

df_prediction_copy.tweet_id = df_prediction_copy.tweet_id.astype('int64')
df_prediction_copy.img_num = df_prediction_copy.img_num.astype('int')

dict_bool = {'True': True, 'False': False}

df_prediction_copy.p1_dog = df_prediction_copy.p1_dog.map(dict_bool)
df_prediction_copy.p2_dog = df_prediction_copy.p2_dog.map(dict_bool)
df_prediction_copy.p3_dog = df_prediction_copy.p3_dog.map(dict_bool)

df_prediction_copy.p1_conf = df_prediction_copy.p1_conf.astype('float')
df_prediction_copy.p2_conf = df_prediction_copy.p2_conf.astype('float')
df_prediction_copy.p3_conf = df_prediction_copy.p3_conf.astype('float')

#### Testar

In [271]:
df_wrd_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         2356 non-null int64
in_reply_to_user_id           2356 non-null int64
timestamp                     2356 non-null datetime64[ns]
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           2356 non-null int64
retweeted_status_user_id      2356 non-null int64
retweeted_status_timestamp    181 non-null datetime64[ns]
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: datetime64

In [272]:
df_prediction_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int32
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int32(1), int64(1), object(4)
memory usage: 144.0+ KB


#### Definir

- As colunas doggo, floofer, pupper, puppo deveriam ser do tipo **bool**

#### Programar

In [273]:
df_wrd_copy.doggo = df_wrd_copy.doggo.str.replace('doggo', 'True')
df_wrd_copy.doggo = df_wrd_copy.doggo.str.replace('None', 'False')
df_wrd_copy.doggo = df_wrd_copy.doggo.map(dict_bool)

df_wrd_copy.floofer = df_wrd_copy.floofer.str.replace('floofer', 'True')
df_wrd_copy.floofer = df_wrd_copy.floofer.str.replace('None', 'False')
df_wrd_copy.floofer = df_wrd_copy.floofer.map(dict_bool)

df_wrd_copy.pupper = df_wrd_copy.pupper.str.replace('pupper', 'True')
df_wrd_copy.pupper = df_wrd_copy.pupper.str.replace('None', 'False')
df_wrd_copy.pupper = df_wrd_copy.pupper.map(dict_bool)

df_wrd_copy.puppo = df_wrd_copy.puppo.str.replace('puppo', 'True')
df_wrd_copy.puppo = df_wrd_copy.puppo.str.replace('None', 'False')
df_wrd_copy.puppo = df_wrd_copy.puppo.map(dict_bool)


#### Testar

In [275]:
df_wrd_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         2356 non-null int64
in_reply_to_user_id           2356 non-null int64
timestamp                     2356 non-null datetime64[ns]
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           2356 non-null int64
retweeted_status_user_id      2356 non-null int64
retweeted_status_timestamp    181 non-null datetime64[ns]
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null bool
floofer                       2356 non-null bool
pupper                        2356 non-null bool
puppo                         2356 non-null bool
dtypes: bool(4), datetime6

#### Definir

- As colunas rating_numerator e rating_denomitator possuem outliers significativos levando em conta o máximo, a média e o percentis.

#### Programar

In [152]:
#coluna rating_denominator deve ser 10, pela especificação
#coluna rating_numeretor possui 26 casos maior que 15
df_wrd_copy[df_wrd_copy.rating_numerator > 15].rating_numerator

55        17
188      420
189      666
290      182
313      960
340       75
433       84
516       24
695       75
763       27
902      165
979     1776
1120     204
1202      50
1228      99
1254      80
1274      45
1351      60
1433      44
1634     143
1635     121
1663      20
1712      26
1779     144
1843      88
2074     420
Name: rating_numerator, dtype: int64

In [154]:
df_wrd_copy = df_wrd_copy[df_wrd_copy.rating_numerator <= 15][df_wrd_copy.rating_denominator == 10]

#### Testar

In [155]:
df_wrd_copy.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.323000e+03,2323.000000,2323.000000,2323.000000,2323.000000,2323.000000,2323.0
mean,7.427125e+17,0.029703,0.029703,0.077056,0.077056,10.704262,10.0
std,6.860718e+16,0.169803,0.169803,0.266737,0.266737,2.176318,0.0
min,6.660209e+17,0.000000,0.000000,0.000000,0.000000,0.000000,10.0
25%,6.783378e+17,0.000000,0.000000,0.000000,0.000000,10.000000,10.0
50%,7.195514e+17,0.000000,0.000000,0.000000,0.000000,11.000000,10.0
75%,7.993029e+17,0.000000,0.000000,0.000000,0.000000,12.000000,10.0
max,8.924206e+17,1.000000,1.000000,1.000000,1.000000,15.000000,10.0


#### Definir

- O formato das colunas p1, p2, p3 deveriam ser padronizados

#### Programar

In [160]:
#Começar com letra maiúscula
#Separação com Espaço

df_prediction_copy.p1 = df_prediction_copy.p1.str.replace('-', ' ')
df_prediction_copy.p1 = df_prediction_copy.p1.str.replace('_', ' ')
df_prediction_copy.p1 = df_prediction_copy.p1.str.title()
df_prediction_copy.p2 = df_prediction_copy.p1.str.replace('-', ' ')
df_prediction_copy.p2 = df_prediction_copy.p1.str.replace('_', ' ')
df_prediction_copy.p2 = df_prediction_copy.p1.str.title() 
df_prediction_copy.p3 = df_prediction_copy.p1.str.replace('-', ' ')
df_prediction_copy.p3 = df_prediction_copy.p1.str.replace('_', ' ')
df_prediction_copy.p3 = df_prediction_copy.p1.str.title()

#### Testar

In [177]:
df_prediction_copy.p1.unique()

array(['Welsh Springer Spaniel', 'Redbone', 'German Shepherd',
       'Rhodesian Ridgeback', 'Miniature Pinscher',
       'Bernese Mountain Dog', 'Box Turtle', 'Chow', 'Shopping Cart',
       'Miniature Poodle', 'Golden Retriever', 'Gordon Setter',
       'Walker Hound', 'Pug', 'Bloodhound', 'Lhasa', 'English Setter',
       'Hen', 'Desktop Computer', 'Italian Greyhound', 'Maltese Dog',
       'Three Toed Sloth', 'Ox', 'Malamute', 'Guinea Pig',
       'Soft Coated Wheaten Terrier', 'Chihuahua',
       'Black And Tan Coonhound', 'Coho', 'Toy Terrier',
       'Blenheim Spaniel', 'Pembroke', 'Llama',
       'Chesapeake Bay Retriever', 'Curly Coated Retriever', 'Dalmatian',
       'Ibizan Hound', 'Border Collie', 'Labrador Retriever', 'Seat Belt',
       'Snail', 'Miniature Schnauzer', 'Airedale', 'Triceratops', 'Swab',
       'Hay', 'Hyena', 'Jigsaw Puzzle', 'West Highland White Terrier',
       'Toy Poodle', 'Giant Schnauzer', 'Vizsla', 'Vacuum', 'Rottweiler',
       'Siberian Husky', 'T

#### Definir

- O nome das colunas timestamp, retweeted_status_timestamp deveriam ser mais intuitivo

#### Programar

In [276]:
df_wrd_copy.rename(columns={'timestamp': 'created_at', 'retweeted_status_timestamp': 'retweeted_at'}, inplace=True)

#### Testar

In [277]:
df_wrd_copy.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,created_at,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_at,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,0,0,2017-08-01 16:23:56,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,0,0,NaT,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,False,False,False,False
1,892177421306343426,0,0,2017-08-01 00:17:27,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,0,0,NaT,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,False,False,False,False
2,891815181378084864,0,0,2017-07-31 00:18:03,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,0,0,NaT,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,False,False,False,False
3,891689557279858688,0,0,2017-07-30 15:58:51,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,0,0,NaT,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,False,False,False,False
4,891327558926688256,0,0,2017-07-29 16:00:24,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,0,0,NaT,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,False,False,False,False


#### Definir

- O nome das colunas jpg_url, img_num, p1, p1_conf, p1_dog, p2, p2_conf, p2_dog, p3, p3_conf, p3_dog deveria ser mais intuitivo

#### Programar

In [279]:
df_prediction_copy.rename(columns={'jpg_url': 'image_url', 'img_num': 'image_count', 
                                   'p1': 'first_prediction', 'p1_conf': 'first_confidence', 'p1_dog': 'first_isdog',
                                   'p2': 'second_prediction', 'p2_conf': 'second_confidence', 'p2_dog': 'second_isdog',
                                   'p3': 'third_prediction', 'p3_conf': 'third_confidence', 'p3_dog': 'third_isdog'
                                  }, inplace=True)

#### Testar

In [280]:
df_prediction_copy.head()

,tweet_id,image_url,image_count,first_prediction,first_confidence,first_isdog,second_prediction,second_confidence,second_isdog,third_prediction,third_confidence,third_isdog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
